<a href="https://colab.research.google.com/github/ian-byrne/MADSmilestone2/blob/main/Transfer_Learning_Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/ian-byrne/MADSmilestone2.git

# Change directory into cloned repo
%cd MADSmilestone2

# List repo contents
!ls

In [ ]:
!pip install torchviz

In [ ]:
!pip install boto3 torchmetrics
from tqdm import tqdm
import boto3
import io
import logging
import os
import ast
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import Loading.load_data as ld


# Pytroch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchmetrics
from torch.autograd import Variable
import botocore

import tempfile


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [144]:
path = '/content/MADSmilestone2/Data/Dictionaries/dementia_label_dicts/train_ai.txt'
cust_file = open(path, "r")
#print(cust_file.readline())
contents = cust_file.read() 
dictionarytr = ast.literal_eval(contents)
cust_file.close()

path1 = '/content/MADSmilestone2/Data/Dictionaries/dementia_label_dicts/val_dict_nhat.txt'
cust_file = open(path1, "r")
#print(cust_file.readline())
contents = cust_file.read()
dictionaryv = ast.literal_eval(contents)
cust_file.close()

path2 = '/content/MADSmilestone2/Data/Dictionaries/dementia_label_dicts/test_dict_nhat.txt'
cust_file = open(path2, "r")
#print(cust_file.readline())
contents = cust_file.read()
dictionaryts = ast.literal_eval(contents)
cust_file.close()

In [145]:
# Define some of the loader variables
train_batch_size = 1
val_batch = 4
test_batch = 1
#rgb_val = None
rnd = 7

normalize_ = True
numpy_ = False

In [ ]:
!pip install graphviz
!pip install hiddenlayer
import torch
import torchvision.models
import hiddenlayer as hl


# Build HiddenLayer graph
# Jupyter Notebook renders it automatically
hl.build_graph(model.to(device), torch.zeros([512, 662]).to(device))

In [ ]:
class ResizedClocks():
    # Resized clock drawing dataset

    def __init__(self, round, round_labels, rgb=None, transform=None, images_path=None):

        # Args:
        # round (int): Round to grab images from.
        # values (list of tuples): Corresponding values for the round.

        self.round = round
        self.vals = round_labels
        self.images_path = images_path
        self.transform = transform
        self.rgb = rgb

    def __len__(self):
        return len(self.vals)

    def __getitem__(self, idx):
        spid = self.vals[idx][0]
        label = torch.tensor(int(self.vals[idx][1]))
        
        # Construct the image path
        img_path = os.path.join(self.images_path, f"{spid}.tif")
        
        try:
            # Load image directly from the path
            im = Image.open(img_path)

            if self.rgb:
                #print('rgb')
                gray = im.convert('RGB')
            
            else:
                #print('gray')
                gray = im.convert('1')

            resized = gray.resize((160, 207))  # 284, 368))#(2560, 3312))
            # resized = gray.resize((512, 662))
            im_arr = np.float32(np.array(resized))  # .astype(float)

            if self.transform:
                im_arr = self.transform(im_arr)

            return im_arr, label

        except (FileNotFoundError, IOError) as e:
            # Return None if image file is not found or corrupted
            return None
def collate_fn(batch):
  """From pytorch - way to bypass corrupt or non-existent data"""
  batch = list(filter(lambda x: x is not None, batch))
  return torch.utils.data.dataloader.default_collate(batch)
# initialize transformation: data to tensor and normalize
# Could probably resize using torch.transforms
if normalize_ == "True":
    processes = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
    rgb_val = "True"
else:
    processes = transforms.ToTensor()
    rgb_val = None



# Define which round to get data from
if numpy_ == False:
  # Get the data from S3 and perform transformations
  train_set = ResizedClocks(rnd, dictionarytr[rnd], transform=processes, rgb=rgb_val, images_path=images_path)
  val_set = ResizedClocks(rnd, dictionaryv[rnd], transform=processes, rgb=rgb_val, images_path=images_path)
  test_set = ResizedClocks(rnd, dictionaryts[rnd], transform=processes, rgb=rgb_val, images_path=images_path)


# Define Dataloaders for the network
train_loader = torch.utils.data.DataLoader(train_set, batch_size = train_batch_size, shuffle = True, num_workers = 6, collate_fn=collate_fn) 
validate_loader = torch.utils.data.DataLoader(val_set, batch_size = val_batch, shuffle = True, num_workers = 6, collate_fn=collate_fn) #64, 8,1


# For round 10, there are some corrupt data that when batched at size 1 is not taken
# care of by the collate function, but Nonechucks library skips the missing data and 
# moves on, replacing that missing data index with the next piece of data
# could probably just use this in place of collate for all the loading
if rnd == 10:
  !pip install nonechucks
  import nonechucks as nc
  test_set_safe = nc.SafeDataset(test_set)
  test_loader = torch.utils.data.DataLoader(test_set_safe, batch_size = test_batch, shuffle = False)

else:
  test_loader = torch.utils.data.DataLoader(test_set, batch_size = test_batch, shuffle = False, collate_fn=collate_fn)

In [ ]:
#original size: 2560, 3312
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    
    # without considering batch size: Input shape : (None,368, 284, 1) , parameters: (3*3*1*16+16) = 160
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 16, # one input channel gray scale, 16 filters out
                            kernel_size = 3, stride = 1, padding = 1) #Out:(None,386, 284, 16). ### TRY kernel 7x7 padding 3
    self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, 
                          kernel_size = 3, stride = 1, padding = 1) #params: (3*3*16*32+32) = 4640                        
    self.pool1 = nn.MaxPool2d(2, 2) #Out: (None, 184, 142, 32) 
    self.bn1 = nn.BatchNorm2d(32)

    self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, 
                          kernel_size = 3, stride = 1, padding = 1) #params: (3*3*16*32+32) = 4640    
    self.conv4 = nn.Conv2d(in_channels = 64, out_channels = 64, 
                          kernel_size = 3, stride = 1, padding = 1) # params: (3*3*32*32+32) = 9248                    
    self.pool2 = nn.MaxPool2d(2, 2) #Output shape = (None, 92, 71, 64) 
    self.bn2 = nn.BatchNorm2d(64)  

    #self.conv5 = nn.Conv2d(in_channels = 64, out_channels = 128, 
                          #kernel_size = 3, stride = 1, padding = 1) # params: (3*3*32*32+32) = 9248 
    self.conv6 = nn.Conv2d(in_channels = 64, out_channels = 128, 
                          kernel_size = 3, stride = 1, padding = 1) # params: (3*3*32*32+32) = 9248
    self.pool3 = nn.MaxPool2d(2, 2) #Output shape = (None, 46, 35, 128) 
    self.bn3 = nn.BatchNorm2d(128)
    self.do2 = nn.Dropout(0.3)
                                   
                             
    # Fully connected layer
    self.fc1 = nn.Linear(128*64*82,60) #most recent original size of: 512, 662 -->64 x 82
    self.do3 = nn.Dropout(0.4) #40 % probability  
    #self.fc3 = nn.Linear(60, 30)
    self.fc2 = nn.Linear(60, 3) # left with 3 for the three classes                     



  def forward(self, x):
    x = self.bn1(self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))))
    x = self.bn2(self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))))
    #x = self.bn3(self.pool3(F.relu(self.conv6(F.relu(self.conv5(x))))))
    x = self.bn3(self.pool3(F.relu(self.conv6((x)))))
    x = self.do2(x)
    x = x.view(x.size(0),128*64*82)
    x = F.relu(self.fc1(x))
    x = self.do3(x)
    x = self.fc2(x)
    return x              

# Resnet50 + KMeans

In [147]:
import torch
import torchvision.models as models
import torch.nn as nn
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

mod = 'res50'

if mod == 'res':
  model_ext = '4_hat'
  mPATH = '/content/gdrive/MyDrive/Colab Notebooks/Models/cnn_512_662.model{}'.format(model_ext)
  model = models.resnet50()#'pretrained')
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, 3)
  model.load_state_dict(torch.load(mPATH))
  model.to(device)
  print("Res half data")

if mod == 'conv':
  model_ext = '4_fix'
  mPATH = '/content/gdrive/MyDrive/Colab Notebooks/Models/cnn_512_662.model{}'.format(model_ext)
  model = ConvNet()
  #model.load_state_dict(torch.load(mPATH, map_location=torch.device('cpu')))
  model.load_state_dict(torch.load(mPATH))
  model.to(device)
  print("conv")

if mod == 'res50':
  model = models.resnet50(pretrained = True)
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, 3)
  model = model.to(device)
  print("res 50 pretrained")

res 50 pretrained


In [ ]:
model

In [148]:
import torch.nn as nn
# Remove prediction layer (last fc layer)
model_1 = nn.Sequential(*list(model.children())[:-1])

In [ ]:
# Use model for evaluation
model_1.eval()

In [ ]:
X = []

for x, lb in tqdm(test_loader):
  x = x.to(device)
  preds = model_1(x)
  X.append(preds.cpu())

In [53]:
X[0].shape

torch.Size([1, 2048, 1, 1])

In [54]:
x_ = np.array([t.cpu().detach().numpy() for t in X])
#x_ = np.array([t for t in X])
print("shape: ", x_.shape)

shape:  (254, 1, 2048, 1, 1)


In [55]:
w = x_.reshape(x_.shape[0], -1)
w.shape

(254, 2048)

In [56]:
from sklearn.decomposition import PCA
# Make an instance of the Model
variance = 0.98 #The higher the explained variance the more accurate the model will remain, but more dimensions will be present
pca = PCA(variance)

pca.fit(w) #fit the data according to our PCA instance
print("Number of components before PCA  = " + str(w.shape[1]))
print("Number of components after PCA 0.98 = " + str(pca.n_components_)) 
#dimension reduced from 784


Number of components before PCA  = 2048
Number of components after PCA 0.98 = 114


In [57]:
w_pca = pca.transform(w)
print("Dim after PCA: ", w_pca.shape)

Dim after PCA:  (254, 114)


In [61]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
sil_score = []
kl = []
k_max = 9

for k in range(2, k_max+1):
  kmeans2 = KMeans(n_clusters = k).fit(w)#w)
  labels = kmeans2.labels_
  sil_score.append(silhouette_score(w, labels, metric = 'euclidean'))
  kl.append(k)

In [ ]:
sil_name = '50res_fit7_hatsil.png'
plt.plot(kl, sil_score)
plt.ylabel('Silhoutte Score')
plt.xlabel('Best K for KMeans')
#plt.title('PCA dim from 2048 to 114')
plt.title('fit to round 7')
plt.suptitle('ResNet pretrained on ImageNet')
#plt.title('Res trained on 50% dataset and fit to Round 7')
plt.savefig('/content/gdrive/MyDrive/Colab Notebooks/model_charts/{}'.format(sil_name))
plt.show()

# Apply optimal K with clustering for labeling

In [ ]:
x_trainK = []
y_train = []
# Pulling in data from round 3 to predict labels
# Since we predicted optimal clusters with resnet50 trained on rounds 1, 5, 6, 7
for x, lb in tqdm(train_loader):
  x_trainK.append(x)
  y_train.append(lb)
  #preds = model(x)
  #make_dot(preds.mean(), params=dict(model.named_parameters()), show_attrs=True, show_saved=True)

In [ ]:
#y_train_ = np.array([t.numpy() for t in y_train])
y_train_ = np.array(y_train)
print("shape: ", (y_train_).shape)
print(type(y_train_))
y_train_

In [ ]:
x_train_ = np.array([t.numpy() for t in x_trainK])
print("shape: ", x_train_.shape)
print(type(x_train_))
x_train_[:1]

In [ ]:
X_train = x_train_.reshape(x_train_.shape[0], -1) # reshaped to # samples and flattened dims
X_train.shape

In [133]:
from sklearn.decomposition import PCA
variance = 0.98 #The higher the explained variance the more accurate the model will remain, but more dimensions will be present
pca = PCA(variance)
pca.fit(X_train)
pct = pca.transform(X_train)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 3, random_state = 0).fit(X_train)
kmeans.labels_

In [ ]:
y_train_

In [ ]:
kmeans.labels_.shape

In [ ]:
clusters = kmeans.fit_predict(X_train)
kmeans.cluster_centers_.shape

In [ ]:
correct_pred = (clusters == y_train_)
  #acc = correct_pred.sum() / len(correct_pred)
print(correct_pred)
print(correct_pred.sum())
len(correct_pred)
acc = correct_pred.sum()/len(correct_pred)
acc

Using NHATS label for round 7 fit on Pretrained ReNet50 on ImageNet only using 3 clusters was 79% accurate compared to cluster labels 


In [ ]:
#y_pred = kmeans.predict(X_train)
fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(111, projection='3d')
plt.scatter(X_train[clusters == 0,0],X_train[clusters == 0,1],s = 50, c = 'green', label = "cluster 1")
plt.scatter(X_train[clusters == 1,0],X_train[clusters == 1,1],s = 50, c = 'blue', label = "cluster 2")
plt.scatter(X_train[clusters == 2,0],X_train[clusters == 2,1],s = 50, c = 'pink', label = "cluster 3")
plt.scatter(kmeans.cluster_centers_[:,0],kmeans.cluster_centers_[:,1], s = 100, c = "red", label = "centroids")
plt.legend()
plt.show()

# With Keras

In [ ]:
keras_x = []

for xk, _ in tqdm(test_loader):
  keras_x.append(xk)

In [ ]:
print("shape of keras_x: ", keras_x[0].shape)
karr = np.array([t.cpu().detach().numpy() for t in keras_x])
print("shape: ", karr.shape)
im1= karr.reshape(len(keras_x), -1, 3)
print('shape: ', im1.shape)
print("dtype: ", im1.dtype)
print("type: ", type(im1))

img1 = Image.fromarray(im1, 'RGB')
im_arr2 = np.float32(np.array(img1))
im_arr2.shape

In [ ]:
import tensorflow as tf 
model_keras = tf.keras.applications.MobileNetV2(include_top=False,
weights= 'imagenet', input_shape=(207, 160, 3))

print("Image shape before model: ", im_arr2.shape)
predictions1 = model_keras.predict(im_arr2.reshape(-1, 207, 160, 3))
print("preds shape after model: ", predictions1.shape)

In [ ]:
pred_images1 = predictions1.reshape(im_arr2.shape[0], -1)
print("Shape of prediction images: ", pred_images1.shape)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
sil_score = []
kl = []
k_max = 9

for k in range(2, k_max+1):
  kmeans2 = KMeans(n_clusters = k).fit(pred_images1)
  labels = kmeans2.labels_
  sil_score.append(silhouette_score(pred_images1, labels, metric = 'euclidean'))
  kl.append(k)

In [ ]:
plt.plot(kl, sil_score)
plt.ylabel('Silhoutte Score')
plt.ylabel('K')
plt.show()